In [1]:
import os
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
import yaml
import pickle as pkl
import subprocess

In [2]:
from tensorflow.keras.regularizers import L1L2


from src.data_preparation import load_data
from src.data_preparation.blocking_time_series_split import BlockingTimeSeriesSplit 
from src.data_preparation import mdl_dataset

from src.modelling import mdl_input_dico  # input variables class
from src.modelling import mdl_params  # parameters class
from src.modelling import mdl_history

# from src.modelling import model_cnn
from src.modelling import model_lstm
from src.modelling import super_model_dl

from src.visualization import visualize_pca
from src.visualization import mdl_introspect
from src.visualization import mdl_ypred_PCA

from src.utils import reload_config
from src.utils import tardisml_utils

2024-03-18 15:20:10.607450: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-18 15:20:10.804695: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /LUMI_CONTAINER/env1/lib:/LUMI_CONTAINER/env1/lib64:/opt/cray/pe/papi/7.0.1.1/lib64:/opt/cray/libfabric/1.15.2.0/lib64:/.singularity.d/libs
2024-03-18 15:20:10.804746: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-03-18 15:20:11.745627: W tensorflow/compiler/xla/stream_executor/platform/default

In [3]:
import tensorflow as tf

In [4]:
ngpu = len(tf.config.list_physical_devices('GPU'))
print("\nNum GPUs Available: ", ngpu)


Num GPUs Available:  0


2024-03-18 15:03:39.624978: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /LUMI_CONTAINER/env1/lib:/LUMI_CONTAINER/env1/lib64:/opt/cray/pe/papi/7.0.1.1/lib64:/opt/cray/libfabric/1.15.2.0/lib64:/.singularity.d/libs
2024-03-18 15:03:39.625024: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2024-03-18 15:03:39.625060: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (uan03): /proc/driver/nvidia/version does not exist


In [5]:
print(f'GPU device name:{tf.test.gpu_device_name()}')

2024-03-18 15:03:39.707495: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


GPU device name:


--------------------------

In [7]:
# add argument to pass config file
# import sys
# narg = len(sys.argv)  # number of arguments passed
# if narg > 1:
#     file_config = sys.argv[1]
# else:
file_config = '../config/config_to_jobs/config_LSTM_no_bias_0wk.yaml'

rootdir = tardisml_utils.get_rootdir()
# file_config = '../config/data_proc_full.yaml'
conf = reload_config.Config(file_config, rootdir=rootdir, verbose=1)

Config file found: ../config/config_to_jobs/config_LSTM_no_bias_0wk.yaml
PCA results in: /scratch/project_465000269/edelleo1/Leo/results/pca_i100-550_j300-629
Config file updated 'pca_dir': ../config/config_to_jobs/config_LSTM_no_bias_0wk.yaml
Results in: /scratch/project_465000269/edelleo1/Leo/results/lstm_230413-162021
Folder created

Subfolder created: /scratch/project_465000269/edelleo1/Leo/results/lstm_230413-162021/ml/
Subfolder created: /scratch/project_465000269/edelleo1/Leo/results/lstm_230413-162021/figures/
Config file updated 'results_dir': ../config/config_to_jobs/config_LSTM_no_bias_0wk.yaml
Config folders updated.
Config copied to: /scratch/project_465000269/edelleo1/Leo/results/lstm_230413-162021
Config file found: /scratch/project_465000269/edelleo1/Leo/results/lstm_230413-162021/config_LSTM_no_bias_0wk.yaml
Default config file is now the copied following one:
/scratch/project_465000269/edelleo1/Leo/results/lstm_230413-162021/config_LSTM_no_bias_0wk.yaml


In [4]:
ipath = 'Leo/results/lstm_240130-173040/'
rootdir = tardisml_utils.get_rootdir()


# ---------------------------
# Load config file
# ---------------------------

file_config = f'{rootdir}{ipath}'

conf = reload_config.Config(file_config, verbose=1)

Config file found: /scratch/project_465000269/edelleo1/Leo/results/lstm_240130-173040/config_h4_EPO.yaml


In [5]:
ds = mdl_dataset.Dataset(conf, setup=conf.setup, non_assimilated=conf.non_ass)


Initialisation dataset...
History loaded from config file: /scratch/project_465000269/edelleo1/Leo/results/lstm_240130-173040/config_h4_EPO.yaml
Loading data...
	Loading sia...
	Retrieve siconc
	Retrieve sisnthick
	Retrieve zos
	Retrieve vxsi
	Retrieve vysi
Scaling...
	Scalers saved as /scratch/project_465000269/edelleo1/Leo/results/lstm_240130-173040/ml/scalers.pkl
Size of the training set:    2581 days
Size of the validation set:   645 days
Size of the test set:        1036 days
Config file updated 'ntrain': /scratch/project_465000269/edelleo1/Leo/results/lstm_240130-173040/config_h4_EPO.yaml
Config file updated 'nval': /scratch/project_465000269/edelleo1/Leo/results/lstm_240130-173040/config_h4_EPO.yaml
Config file updated 'ntest': /scratch/project_465000269/edelleo1/Leo/results/lstm_240130-173040/config_h4_EPO.yaml
Dataset Format for lstm
Config file updated 'needfutur': /scratch/project_465000269/edelleo1/Leo/results/lstm_240130-173040/config_h4_EPO.yaml
Config file updated 'need

In [ ]:
m1 = model_lstm.ModelLSTM(ds, timesteps=ds.dataset['X'].shape[1], features=ds.dataset['X'].shape[2])


# compile same architecture as during training
m1.compile_models(npca=conf.n_comp['tp'])

In [7]:
ifolder_pattern = f"model_weights_{conf.ml_name}_{conf.n_comp['tp']}N"  # 8 from config file

# load weights
m1.load_model_weights(ipath=f'{rootdir}{ipath}ml/', ifolder_pattern=ifolder_pattern)

Loading models...
Loaded: /scratch/project_465000269/edelleo1/Leo/results/lstm_240130-173040/ml/model_weights_LSTM3_bk_8N_PC0
Loaded: /scratch/project_465000269/edelleo1/Leo/results/lstm_240130-173040/ml/model_weights_LSTM3_bk_8N_PC1
Loaded: /scratch/project_465000269/edelleo1/Leo/results/lstm_240130-173040/ml/model_weights_LSTM3_bk_8N_PC2
Loaded: /scratch/project_465000269/edelleo1/Leo/results/lstm_240130-173040/ml/model_weights_LSTM3_bk_8N_PC3
Loaded: /scratch/project_465000269/edelleo1/Leo/results/lstm_240130-173040/ml/model_weights_LSTM3_bk_8N_PC4
Loaded: /scratch/project_465000269/edelleo1/Leo/results/lstm_240130-173040/ml/model_weights_LSTM3_bk_8N_PC5
Loaded: /scratch/project_465000269/edelleo1/Leo/results/lstm_240130-173040/ml/model_weights_LSTM3_bk_8N_PC6
Loaded: /scratch/project_465000269/edelleo1/Leo/results/lstm_240130-173040/ml/model_weights_LSTM3_bk_8N_PC7


In [8]:
# ---------------------------------------------------
#                 Defining parameters
# ---------------------------------------------------

H = [1,8,15,22]


new_hist = mdl_history.History(conf)
new_hist.set_zeros()
new_hist.set_SIT_history('keras', H)

# var_to_keep = ['SITf t+0', 'sisnthick t+0', '2T t+0', '2T t+8', 'MSL t+0', 'TP t+0', 'TP t+8', 'siconc t+0'] #tradi
# var_to_keep = ['SITf t+0', 'sisnthick t+0', '2T t+0', 'MSL t+0', 'TP t+0', 'siconc t+0']
var_to_keep = ['SITf t+0', 'sisnthick t+0', '2T t+0', 'MSL t+0', 'TP t+0', 'siconc t+0', 'SIA t+0']

In [8]:
from src.data_preparation import load_data

In [10]:
reload(mdl_dataset)

<module 'src.data_preparation.mdl_dataset' from '/cluster/home/leoede/tardis/tardis-ml/src/data_preparation/mdl_dataset.py'>

In [9]:
# ---------------------------------------------------
#                 Loading data
# ---------------------------------------------------

#ds = mdl_dataset.Dataset(conf, setup='no_bias', history=new_hist, var_to_keep=var_to_keep)
ds = mdl_dataset.Dataset(conf, setup=conf.setup, history=new_hist, var_to_keep=var_to_keep)

# ds_nn = [ds.dataset, 'change le format stp']
# ntvt = (ds.ntrain, ds.nval, ds.ntest)

# -----------------------------------------------------------


Initialisation dataset...
Loading data...


/LUMI_TYKKY_KyB8utx/env1/lib/python3.9/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator PCA from version 1.1.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/LUMI_TYKKY_KyB8utx/env1/lib/python3.9/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator PCA from version 1.1.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Loading sia...
SIA off ! !!!!!!! just to be able to finish poster
	Retrieve siconc


/LUMI_TYKKY_KyB8utx/env1/lib/python3.9/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator PCA from version 1.1.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


	Retrieve sisnthick


...

	Retrieve vysi


/LUMI_TYKKY_KyB8utx/env1/lib/python3.9/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator PCA from version 1.1.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Loading forcing values...
Loading pca...


/LUMI_TYKKY_KyB8utx/env1/lib/python3.9/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator PCA from version 1.1.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Apply land/ocean mask...
Retrieve PCs and EOFs
Adding sia
New variable selection:
  ['SITf t+0 PC0' 'SITf t+0 PC1' 'SITf t+0 PC2' 'SITf t+0 PC3'
 'SITf t+0 PC4' 'SITf t+0 PC5' 'SITf t+0 PC6' 'SITf t+0 PC7'
 'sisnthick t+0 PC0' 'sisnthick t+0 PC1' 'sisnthick t+0 PC2'
 'sisnthick t+0 PC3' '2T t+0 PC0' '2T t+0 PC1' '2T t+0 PC2' '2T t+0 PC3'
 'MSL t+0 PC0' 'MSL t+0 PC1' 'MSL t+0 PC2' 'MSL t+0 PC3' 'TP t+0 PC0'
 'TP t+0 PC1' 'TP t+0 PC2' 'TP t+0 PC3' 'siconc t+0 PC0' 'siconc t+0 PC1'
 'siconc t+0 PC2' 'siconc t+0 PC3']
Variables updated: do not forget to execute self.format_for_dl() if Deep Learning
Size of the training set:    1944 days
Size of the validation set:   448 days
Size of the test set:         600 days
Config file updated 'ntrain': /scratch/project_465000269/edelleo1/Leo/results/lstm_230413-162021/config_LSTM_no_bias_0wk.yaml
Config file updated 'nval': /scratch/project_465000269/edelleo1/Leo/results/lstm_230413-162021/config_LSTM_no_bias_0wk.yaml
Config file updated 'ntest': /s

In [10]:
conf.epochs = 6

In [42]:
reload(super_model_dl)

<module 'src.modelling.super_model_dl' from '/cluster/home/leoede/tardis/tardis-ml/src/modelling/super_model_dl.py'>

In [12]:
from importlib import reload

In [16]:
reload(super_model_dl)

<module 'src.modelling.super_model_dl' from '/users/edelleo1/tardis/tardis-ml/src/modelling/super_model_dl.py'>

In [17]:
reload(model_lstm)

<module 'src.modelling.model_lstm' from '/users/edelleo1/tardis/tardis-ml/src/modelling/model_lstm.py'>

In [19]:
regularizers = [L1L2(l1=0, l2=0)]  # [L1L2(l1=0, l2=0), L1L2(l1=0.01, l2=0), L1L2(l1=0, l2=0.01), L1L2(l1=0.1, l2=0.1)]

n_components = ds.config.n_comp['tp']


ireg = 0
reg = regularizers[ireg]
suffixe = f'_reg{ireg}'
            
with tf.device("CPU"):

    # n_output = ds[0]['ytrain'].shape[0]
    m4 = model_lstm.ModelLSTM(ds, ds.dataset['Xtrain'].shape[1], ds.dataset['Xtrain'].shape[2], reg=reg, rootdir=ds.config.rootdir, ml_dir=ds.config.ml_dir, fig_dir=ds.config.fig_dir)
    m4.compile_models(npca=n_components)
    m4.fit_multiple(ds.dataset, suffix=suffixe) 
    m4.print_histories()
    m4.predict_multiple(ds.dataset)
    m4.save_prediction()

    # m4.save_model()
    m4.save_model_weights()


    ofile = f'{m4.type}_ypred.png'
    mdl_ypred_PCA.draw(m4, odir=m4.rootdir+m4.fig_dir, savefig=True, showfig=False, ofile=ofile)

input change depending on model !!


2023-04-13 16:50:59.677584: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-04-13 16:50:59.678369: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-04-13 16:50:59.683408: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-04-13 16:50:59.684056: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-04-13 16:50:59.684692: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-04-13 16:50:59.696324: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-04-13 16:50:59.708146: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-04-13 16:50:59.708736: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-04-13 16:50:59.709303: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-04-13 16:50:59

Compiled !
Compiled !
Compiled !
Compiled !
Compiled !
Compiled !
Compiled !
Compiled !
Creation of /scratch/project_465000269/edelleo1/Leo/results/lstm_230413-162021/ml/logs_20230413-165103
Fitting...
Epoch 1/6


2023-04-13 16:51:03.405558: I tensorflow/core/profiler/lib/profiler_session.cc:101] Profiler session initializing.
2023-04-13 16:51:03.405586: I tensorflow/core/profiler/lib/profiler_session.cc:116] Profiler session started.
2023-04-13 16:51:03.405598: I tensorflow/core/profiler/backends/gpu/device_tracer_rocm.cc:911] GpuTracer created.
2023-04-13 16:51:03.405744: I tensorflow/core/profiler/backends/gpu/rocm_tracer.cc:1393] GpuTracer started
2023-04-13 16:51:03.405770: I tensorflow/core/profiler/lib/profiler_session.cc:128] Profiler session tear down.
2023-04-13 16:51:03.405861: I tensorflow/core/profiler/backends/gpu/rocm_tracer.cc:1407] GpuTracer stopped
2023-04-13 16:51:03.411260: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-04-13 16:51:03.419239: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-04-13 16:51:03.425431: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-04-

 24/483 [>.............................] - ETA: 2s - loss: 1571.4824 - r_square: -0.1068 - root_mean_squared_error: 39.6419 

...

62/93 [===================>..........] - ETA: 0s

2023-04-13 16:53:28.270658: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-04-13 16:53:28.297223: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-04-13 16:53:28.323367: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-04-13 16:53:28.347023: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.


93/93 [==============================] - 1s 2ms/step
Prediction finished !! - for lstm_at only
Save prediction as .nc
Ytrue saved as: /scratch/project_465000269/edelleo1/Leo/results/lstm_230413-162021/ml/ypred_LSTM3_bk.nc
Saving models weights...
Saved as: /scratch/project_465000269/edelleo1/Leo/results/lstm_230413-162021/ml/model_weights_LSTM3_bk_8N_PC0
Saved as: /scratch/project_465000269/edelleo1/Leo/results/lstm_230413-162021/ml/model_weights_LSTM3_bk_8N_PC1
Saved as: /scratch/project_465000269/edelleo1/Leo/results/lstm_230413-162021/ml/model_weights_LSTM3_bk_8N_PC2
Saved as: /scratch/project_465000269/edelleo1/Leo/results/lstm_230413-162021/ml/model_weights_LSTM3_bk_8N_PC3
Saved as: /scratch/project_465000269/edelleo1/Leo/results/lstm_230413-162021/ml/model_weights_LSTM3_bk_8N_PC4
Saved as: /scratch/project_465000269/edelleo1/Leo/results/lstm_230413-162021/ml/model_weights_LSTM3_bk_8N_PC5
Saved as: /scratch/project_465000269/edelleo1/Leo/results/lstm_230413-162021/ml/model_weights

2023-04-13 16:53:28.541283: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-04-13 16:53:28.542054: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-04-13 16:53:28.542634: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-04-13 16:53:28.543245: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-04-13 16:53:28.543823: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-04-13 16:53:28.544327: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-04-13 16:53:28.544814: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-04-13 16:53:28.545463: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-04-13 16:53:28.545939: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-04-13 16:53:28

Saved as: /scratch/project_465000269/edelleo1/Leo/results/lstm_230413-162021/figures/LSTM3_bk_ypred.png


# SHAP

# tout bugger: LSTM fonctionne pas avec ces versions de TF et SHAP

In [8]:
import shap

/LUMI_CONTAINER/env1/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
import tensorflow as tf

In [19]:
tf.compat.v1.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [12]:
tf.__version__

'2.11.0'

In [11]:
shap.__version__

'0.45.0'

In [10]:
m1.models['pc0'].summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 7, 32)             54400     
                                                                 
 dropout (Dropout)           (None, 7, 32)             0         
                                                                 
 lstm_1 (LSTM)               (None, 7, 32)             8320      
                                                                 
 dropout_1 (Dropout)         (None, 7, 32)             0         
                                                                 
 lstm_2 (LSTM)               (None, 32)                8320      
                                                                 
 dropout_2 (Dropout)         (None, 32)                0         
                                                                 
 dense (Dense)               (None, 1)                 3

In [13]:
m1.x.shape

AttributeError: 'ModelLSTM' object has no attribute 'x'

In [14]:
ds.dataset['X'].shape

(4143, 7, 392)

In [32]:
2970*2

5940

In [21]:
# DEEP EXPLAINER - works with 3D dataset

explainer = shap.DeepExplainer(m1.models['pc0'], ds.dataset['Xtrain'])

TypeError: Tensors are unhashable (this tensor: KerasTensor(type_spec=TensorSpec(shape=(?, 7, 392), dtype=tf.float32, name='lstm_input'), name='lstm_input', description="created by layer 'lstm_input'")). Instead, use tensor.ref() as the key.

In [16]:
ds.dataset['Xtrain'].shape

(2438, 7, 392)

In [20]:
shap_values = explainer.shap_values(ds.dataset['Xtrain'][:10])

TypeError: Tensors are unhashable (this tensor: KerasTensor(type_spec=TensorSpec(shape=(?, 7, 392), dtype=tf.float32, name='lstm_input'), name='lstm_input', description="created by layer 'lstm_input'")). Instead, use tensor.ref() as the key.